In [ ]:
import pandas as pd
import numpy as np
import re
import string
import csv
import os
import xml.etree.ElementTree as ET

In [ ]:
ns_names = pd.read_csv("ns_hra/ns_disease_names.csv", header =None)

In [ ]:
keywords = ns_names[0].tolist()  
# Names for the diseases of the nervous system in the ICD 10

In [ ]:
print(keywords)

In [ ]:
known_tag = 'Study_Full_Title' 

keyword_with_abbreviations = {kw.lower(): abbreviations.get(kw, '').lower() for kw in keywords}

# Study Title
def title_contains_keyword_or_abbreviation(title_text):
    title_text = title_text.lower()  
    for keyword, abbreviation in keyword_with_abbreviations.items():
        if keyword in title_text or (abbreviation and abbreviation in title_text):
            return True  
    return False  

def extract_next_after_known_tag_and_check_keywords(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        found_known_tag = False
        for elem in root.iter():
            if found_known_tag:
                title_text = elem.text
                if title_text and title_contains_keyword_or_abbreviation(title_text):
                    return True  
                break  
            if elem.tag == known_tag:
                found_known_tag = True

        return False  

    except ET.ParseError as e:
        print(f"Error parsing {xml_file}: {e}")
        return False

matching_files = []

xml_directory = ''

for xml_file in os.listdir(xml_directory):
    if xml_file.endswith(".xml"):
        file_path = os.path.join(xml_directory, xml_file)
        if extract_next_after_known_tag_and_check_keywords(file_path):
            matching_files.append(xml_file)

with open("matching_files.txt", "w") as f:
    for file_name in matching_files:
        f.write(f"{file_name}\n")

print(f"Total matching files: {len(matching_files)}")
